In [ ]:
from file_utils import read_file_in_dir
from dataset_factory import get_datasets
from model_factory import get_model
import torch
import os

In [ ]:
# Specify the experiment name
experiment_name = 'experiment_1'

In [ ]:
config_data = read_file_in_dir("./", f"{experiment_name}.json")
coco_test, vocab, _, _, test_loader = get_datasets(config_data)
model = get_model(config_data, vocab)
state_dict = torch.load(f"./experiment_data/{experiment_name}/best_model.pt")
model.load_state_dict(state_dict)